In [19]:
import os
import io
import csv
import base64
import detectron2
import glob
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# import some common libraries
import numpy as np
import cv2
import torch

import pandas as pd
from tqdm.notebook import tqdm
import json

# Show the image in ipynb
from IPython.display import clear_output, Image, display
import PIL.Image
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [20]:
with open("/data2/zhongkai/VIP/real_world_dataset/random_split/train/image_map.json", 'r') as f:
    maps = json.load(f)
PATH = "/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/"

In [21]:
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image

def doit(raw_image, NUM_OBJECTS):
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        print("Transformed image size: ", image.shape[:2])
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Generate proposals with RPN
        proposals, _ = predictor.model.proposal_generator(images, features, None)
        proposal = proposals[0]
        print('Proposal Boxes size:', proposal.proposal_boxes.tensor.shape)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [x.proposal_boxes for x in proposals]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_attr_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        outputs = FastRCNNOutputs(
            predictor.model.roi_heads.box2box_transform,
            pred_class_logits,
            pred_proposal_deltas,
            proposals,
            predictor.model.roi_heads.smooth_l1_beta,
        )
        probs = outputs.predict_probs()[0]
        boxes = outputs.predict_boxes()[0]
        
        attr_prob = pred_attr_logits[..., :-1].softmax(-1)
        max_attr_prob, max_attr_label = attr_prob.max(-1)
        
        # Note: BUTD uses raw RoI predictions,
        #       we use the predicted boxes instead.
        # boxes = proposal_boxes[0].tensor    
        
        # NMS
        for nms_thresh in np.arange(0.5, 1.0, 0.1):
            instances, ids = fast_rcnn_inference_single_image(
                boxes, probs, image.shape[1:], 
                score_thresh=0.2, nms_thresh=nms_thresh, topk_per_image=NUM_OBJECTS
            )
            if len(ids) == NUM_OBJECTS:
                break
                
        instances = detector_postprocess(instances, raw_height, raw_width)
        roi_features = feature_pooled[ids].detach()
        max_attr_prob = max_attr_prob[ids].detach()
        max_attr_label = max_attr_label[ids].detach()
        instances.attr_scores = max_attr_prob
        instances.attr_classes = max_attr_label
        
        print(instances)
        
        return instances, roi_features


In [22]:
# Load VG Classes
data_path = 'data/genome/1600-400-20'

vg_classes = []
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        vg_classes.append(object.split(',')[0].lower().strip())

MetadataCatalog.get("vg").thing_classes = vg_classes


In [23]:
cfg = get_cfg()
cfg.merge_from_file("../configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml")
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 300
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = 'cuda:1'
# VG Weight
cfg.MODEL.WEIGHTS = "http://nlp.cs.unc.edu/models/faster_rcnn_from_caffe_attr.pkl"
predictor = DefaultPredictor(cfg)

Config '../configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


Modifications for VG in ResNet Backbone (modeling/backbone/resnet.py):
	Using pad 0 in stem max_pool instead of pad 1.

Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401



In [24]:
def generate_feature(image, ids, num_objects=30):
    instances, features = doit(image, num_objects)

    boxes = instances.pred_boxes.tensor.cpu().numpy()
    image_height, image_width = instances.image_size
    feature = features.cpu().numpy()

    box_width = boxes[:, 2] - boxes[:, 0]
    box_height = boxes[:, 3] - boxes[:, 1]
    scaled_width = box_width/image_width
    scaled_height = box_height/image_height
    scaled_x = boxes[:, 0]/image_width
    scaled_y = boxes[:, 1] / image_height
    scaled_width = scaled_width[..., np.newaxis]
    scaled_height = scaled_height[..., np.newaxis]
    scaled_x = scaled_x[..., np.newaxis]
    scaled_y = scaled_y[..., np.newaxis]

    spatial_features = np.concatenate((scaled_x, scaled_y, scaled_x+scaled_width, scaled_y+scaled_height, scaled_width, scaled_height), axis=1)

    full_features = np.concatenate((feature, spatial_features), axis=1)
    fea_base64 = base64.b64encode(full_features).decode('utf-8')
    fea_info = {"num_boxes": boxes.shape[0], "features": fea_base64}

    objects = instances.pred_classes.cpu().numpy()
    conf = instances.scores.cpu().numpy()

    labels = []
    for i in range(len(boxes)):
        labels.append({
            "class": vg_classes[objects[i]],
            "rect": list(boxes[i, :]), 
            "conf": conf[i]
        })
    return {'image_id': ids, 'images': fea_info}, {'image_id': ids, 'list':labels}

In [25]:
def generate_lineidx_file(filein, idxout):
    idxout_tmp = idxout + '.tmp'
    with open(filein, 'r') as tsvin, open(idxout_tmp,'w') as tsvout:
        fsize = os.fstat(tsvin.fileno()).st_size
        fpos = 0
        while fpos!=fsize:
            tsvout.write(str(fpos)+"\n")
            tsvin.readline()
            fpos = tsvin.tell()
    os.rename(idxout_tmp, idxout)

def generate_tsv(feature_path="../test/train.feature.tsv", label_path="../test/train.label.tsv"):
    with open(feature_path, 'a') as tsvfile, open(label_path, 'a') as labeltsvfile:
        writer = csv.DictWriter(tsvfile, delimiter='\t', fieldnames=['image_id', 'images'])
        Labelwriter=csv.DictWriter(labeltsvfile, delimiter='\t', fieldnames=['image_id', 'list'])
        for img_path in glob.glob(f"{PATH}/*.jpg"):
            print(img_path)
            im = cv2.imread(img_path)
            im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            index = img_path.split('/')[-1].split('.')[0]
            print(index)
            rst = generate_feature(im_rgb, ids=int(index))

            writer.writerow(rst[0])
            Labelwriter.writerow(rst[1])
    generate_lineidx_file(feature_path, feature_path.replace("tsv", "lineidx"))
    generate_lineidx_file(label_path, label_path.replace("tsv", "lineidx"))



In [26]:
!rm ../test/train*.tsv

rm: cannot remove '../test/train*.tsv': No such file or directory


In [27]:
generate_tsv()

/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1177.jpg
1177
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([177, 4])
Pooled features size: torch.Size([177, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/193.jpg
193
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([134, 4])
Pooled features size: torch.Size([134, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1678.jpg
1678
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([143, 4])
Pooled features size: torch.Size([143, 2048])
Instances(num_i

815
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([150, 4])
Pooled features size: torch.Size([150, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/212.jpg
212
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([143, 4])
Pooled features size: torch.Size([143, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/835.jpg
835
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([132, 4])
Pooled features size: torch.Size([132, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores,

316
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([144, 4])
Pooled features size: torch.Size([144, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1248.jpg
1248
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([161, 4])
Pooled features size: torch.Size([161, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1209.jpg
1209
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([124, 4])
Pooled features size: torch.Size([124, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, sco

331
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([143, 4])
Pooled features size: torch.Size([143, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1657.jpg
1657
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([165, 4])
Pooled features size: torch.Size([165, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1210.jpg
1210
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([132, 4])
Pooled features size: torch.Size([132, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, sco

1099
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([155, 4])
Pooled features size: torch.Size([155, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1494.jpg
1494
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([117, 4])
Pooled features size: torch.Size([117, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1732.jpg
1732
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([137, 4])
Pooled features size: torch.Size([137, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, sc

...

Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/792.jpg
792
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([129, 4])
Pooled features size: torch.Size([129, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/63.jpg
63
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([145, 4])
Pooled features size: torch.Size([145, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1876.jpg
1876
Original image size:  (2160, 3840)
Transform

Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1186.jpg
1186
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([123, 4])
Pooled features size: torch.Size([123, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/32.jpg
32
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([140, 4])
Pooled features size: torch.Size([140, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1528.jpg
1528
Original image size:  (2160, 3840)
Transfo

Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/397.jpg
397
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([134, 4])
Pooled features size: torch.Size([134, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/898.jpg
898
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([135, 4])
Pooled features size: torch.Size([135, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1422.jpg
1422
Original image size:  (2160, 3840)
Transfo

Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1887.jpg
1887
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([144, 4])
Pooled features size: torch.Size([144, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/584.jpg
584
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([137, 4])
Pooled features size: torch.Size([137, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1084.jpg
1084
Original image size:  (2160, 3840)
Trans

Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1005.jpg
1005
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([124, 4])
Pooled features size: torch.Size([124, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/1483.jpg
1483
Original image size:  (2160, 3840)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([149, 4])
Pooled features size: torch.Size([149, 2048])
Instances(num_instances=30, image_height=2160, image_width=3840, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/real_world_dataset/random_split/train/images/154.jpg
154
Original image size:  (2160, 3840)
Trans